In [1]:
import os
from modules.merge_dataset import merge_dataset
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import pipeline, components

In [2]:
merge_dataset("data/movie.csv", "data/rating.csv")

'Merge dataset success'

In [3]:
PIPELINE_NAME = "movie-recommender-pipeline"

# pipeline inputs
DATA_ROOT = "data/merge"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [4]:
components_args = {
    "data_dir": DATA_ROOT,
    "transform_module": TRANSFORM_MODULE_FILE,
    "tuner_module": TUNER_MODULE_FILE,
    "trainer_module": TRAINER_MODULE_FILE,
    "train_steps": 5000,
    "eval_steps": 2000,
    "serving_model_dir": serving_model_dir,
    "epochs": 5,
}

components = components.init_components(components_args)

pipeline = pipeline.init_pipeline(
    pipeline_root,
    PIPELINE_NAME,
    metadata_path,
    components,
)

BeamDagRunner().run(pipeline)

ERROR:absl:ERROR IN init_components:
module 'tfx.components' has no attribute 'Resolver'
